In [7]:
import findspark
findspark.init('/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession 
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from matplotlib import pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [8]:
rdd = sc.textFile('./csv/ratings_Sports_withID.csv')
rdd = rdd.map(lambda x: x.split(','))
rdd = rdd.filter(lambda x: int(x[3]) < 40000)
rdd = rdd.filter(lambda x: int(x[0]) < 8000)
rdd.take(10)

[[u'1412', u'B00004YVAJ', u'5.0', u'15010', u'A35UHVTCH8150C'],
 [u'1412', u'B00004YVAJ', u'5.0', u'20982', u'A2Q7C95JCR0HV7'],
 [u'1412', u'B00004YVAJ', u'3.0', u'15465', u'A1ET2OJ15PESUA'],
 [u'1412', u'B00004YVAJ', u'5.0', u'21070', u'A2PVOGZSGBKCCN'],
 [u'1412', u'B00004YVAJ', u'5.0', u'23138', u'APMAMP7QL4JJ7'],
 [u'1412', u'B00004YVAJ', u'5.0', u'25468', u'A3CZ2AEU7ONYC8'],
 [u'1412', u'B00004YVAJ', u'4.0', u'3096', u'A1MDHR1E6FH490'],
 [u'1412', u'B00004YVAJ', u'5.0', u'33574', u'AR19W42BFDGJX'],
 [u'1412', u'B00004YVAJ', u'5.0', u'5476', u'AAA4HSDSUM369'],
 [u'1412', u'B00004YVAJ', u'5.0', u'16778', u'A2UHQPRBNDR87J']]

In [9]:
user = rdd.map(lambda x: x[3])
num_user = user.distinct().count()
print num_user

1156


In [10]:
item = rdd.map(lambda x: x[0])
num_item = item.distinct().count()
print num_item

695


In [11]:
def mergedic(x, y):
    x.update(y)
    return x

In [13]:
userd = user.distinct().zipWithIndex()
userd = userd.map(lambda x: {x[0]: x[1] + 1})
user_dic = userd.reduce(mergedic)

In [14]:
itemd = item.distinct().zipWithIndex()
itemd = itemd.map(lambda x: {x[0]: x[1] + 1})
item_dic = itemd.reduce(mergedic)

In [15]:
cb = rdd.map(lambda (item, itemname, rating, user, username): [username, itemname, rating, user_dic[user], item_dic[item]])

In [16]:
cbrdd = cb.map(lambda x: Row(userid = int(x[3]),
                             rating = float(x[2]),                            
                             itemid = int(x[4])))

In [17]:
df = spark.createDataFrame(cbrdd)

In [18]:
tdf = df.groupby('userid').agg({'rating': 'mean'})
df = tdf.join(df, tdf.userid == df.userid).drop(tdf.userid)
df = df.withColumnRenamed('avg(rating)', 'usermean')

In [19]:
tdf = df.groupby('itemid').agg({'rating': 'mean'})
df = tdf.join(df, tdf.itemid == df.itemid).drop(tdf.itemid)
df = df.withColumnRenamed('avg(rating)', 'itemmean')

In [22]:
df.repartition(1).write.csv('tempcsv', sep = ',')

In [23]:
df.show()

+-----------------+--------+------+------+------+
|         itemmean|usermean|itemid|rating|userid|
+-----------------+--------+------+------+------+
|              5.0|     5.0|    26|   5.0|   584|
|              1.0|     1.0|    29|   1.0|   986|
|              5.0|     5.0|   474|   5.0|   181|
|4.476190476190476|     4.0|    65|   4.0|  1038|
|4.476190476190476|     5.0|    65|   5.0|   909|
|4.476190476190476|     5.0|    65|   5.0|   271|
|4.476190476190476|     3.0|    65|   3.0|   865|
|4.476190476190476|     5.0|    65|   5.0|   901|
|4.476190476190476|     5.0|    65|   5.0|   250|
|4.476190476190476|     5.0|    65|   5.0|  1155|
|4.476190476190476|     4.0|    65|   4.0|     3|
|4.476190476190476|     4.0|    65|   4.0|   478|
|4.476190476190476|     5.0|    65|   5.0|   164|
|4.476190476190476|     5.0|    65|   5.0|   791|
|4.476190476190476|     5.0|    65|   5.0|   626|
|4.476190476190476|     5.0|    65|   5.0|   393|
|4.476190476190476|     5.0|    65|   5.0|   291|
